<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("markcrowe-com", "agriculture-data-analytics", "notebooks/notebook-3-03-ml-milk-production.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/markcrowe-com/agriculture-data-analytics/master?filepath=notebooks/notebook-3-03-ml-milk-production.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/markcrowe-com/agriculture-data-analytics/blob/master/notebooks/notebook-3-03-ml-milk-production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

# Objective
       Create the best machine learning model to predict Milk production value in Ireland
       according to the historical data from Central Statistics office CSO.
       AEA01 Value at Current Prices for Output, Input and Income in Agriculture
       Downloaded https://data.cso.ie/table/AEA01

# Contents
    - Read data from Assets folder
    - Split to training / testing sets
    - Scale each set seperatly
    - Run Models
        - Define Hyper parameter tuning Cross Validation Grid or Random Search
        - Random Forest Regressor
        - XGBOOST Regressor
        - ANN
    - Save best model into Pickle file
    - Next step: Deploy selected model on a Streamlit webapp

### Setup

Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
#!pip install tensorflow
#!pip install keras-tuner

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import pickle


# Local
#!pip install -r script/requirements.txt --quiet --user
# Remote option
#!pip install -r https://github.com/markcrowe-com/agriculture-data-analytics/blob/master/notebooks/script/requirements.txt --quiet --user
#import data_analytics.exploratory_data_analysis as eda
#import data_analytics.exploratory_data_analysis_reports as eda_reports



### Load dataframe

In [3]:
df = pd.read_csv("./../artifacts/TA_inputoutputvalue_1990_2021_CSO.csv")
print("data dimensions \n",df.shape)
print()
#print("data column info \n",df.info)
print()
print('Data sample\n',df.sample(5))



data dimensions 
 (32, 57)


Data sample
     Unnamed: 0  Year          UNIT  Agricultural Output at Basic Prices  \
9            9  1999  Euro Million                               5651.4   
28          28  2018  Euro Million                               8663.7   
12          12  2002  Euro Million                               5836.1   
21          21  2011  Euro Million                               6576.2   
30          30  2020  Euro Million                               8908.3   

    All Cereals  All Crops  All Livestock  All Livestock Products  \
9         164.1     1184.3         2067.9                  1438.1   
28        288.4     2126.0         3431.4                  2639.2   
12        141.9     1245.3         2014.8                  1451.0   
21        290.1     1715.9         2644.8                  1891.1   
30        289.5     1942.9         3591.8                  2832.0   

    All Livestock Products - Milk  \
9                          1408.8   
28                

In [4]:
# create score dataframe to store model scores
df_score = pd.DataFrame()
print(df_score)

Empty DataFrame
Columns: []
Index: []


## Production of Milk

In [5]:
## Extract milk production dataset
# drop redundunt columns
df = df.drop('Unnamed: 0',axis = 1)

# extract milk dataset
df_milk = df[['Year',
#              'UNIT',
              'All Livestock Products - Milk',
              'Taxes on Products',
              'Subsidies on Products',
              'Compensation of Employees',
              'Contract Work',
              'Entrepreneurial Income',
              'Factor Income',
              #'Fixed Capital Consumption - Farm Buildings',
              #'Fixed Capital Consumption - Machinery, Equipment, etc',
              #'Interest less FISIM',
              #'Operating Surplus',
              #'Livestock - Cattle',
              #'Livestock - Sheep',
              #'Land Rental',
              #'Intermediate Consumption - Contract Work',
              #'Intermediate Consumption - Crop Protection Products',
              #'Intermediate Consumption - Energy and Lubricants',
              #'Intermediate Consumption - Feeding Stuffs',
              #'Intermediate Consumption - Fertilisers',
              #'Intermediate Consumption - Financial Intermediation Services Indirect',
              #'Intermediate Consumption - Forage Plants',
              #'Intermediate Consumption - Maintenance and Repairs',
              #'Intermediate Consumption - Seeds',
              #'Intermediate Consumption - Services',
              #'Intermediate Consumption - Veterinary Expenses',
              #'Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)',
              #'Intermediate Consumption - Other Goods and Services'
              
             ]]
# Assign year as index
df_milk.set_index('Year',drop=True,inplace=True)

print("Milk production dataset dimenssions \n", df_milk.shape)
print("Milk production dataset Sample \n", df_milk.head())


Milk production dataset dimenssions 
 (32, 7)
Milk production dataset Sample 
       All Livestock Products - Milk  Taxes on Products  Subsidies on Products  \
Year                                                                            
1990                         1316.3               75.0                  408.9   
1991                         1258.9               78.2                  357.3   
1992                         1373.1               79.5                  446.0   
1993                         1439.0               68.0                  466.4   
1994                         1446.2               53.7                  666.0   

      Compensation of Employees  Contract Work  Entrepreneurial Income  \
Year                                                                     
1990                      377.6          180.7                  1577.2   
1991                      362.5          172.0                  1440.2   
1992                      336.8          179.8           

In [6]:
#eda_reports.print_dataframe_analysis_report(df_milk)


### Define 20% Training set 80% Test set

In [7]:
# define target & feature variables

X = df_milk.iloc[:,2:].values
Y = df_milk.iloc[:,1].values.reshape(-1,1)
print('features dimension ',np.shape(X))
print('target dimension ',np.shape(Y))

# impute mean value for NA's
#from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp_mean.fit_transform(X)
Y = imp_mean.fit_transform(Y)


# split train test split 20
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2021)
print()
print('x_train dimension ', X_train.shape)
print('y_train dimension ', Y_train.shape)
print()
print('x_test dimension ', X_test.shape)
print('y_test dimension ', Y_test.shape)

features dimension  (32, 5)
target dimension  (32, 1)

x_train dimension  (25, 5)
y_train dimension  (25, 1)

x_test dimension  (7, 5)
y_test dimension  (7, 1)


### Scale & Transform

In [8]:
# Scale raining set and test set seperatly
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# calculate mean and std of training set 
scaler_x.fit(X_train)
scaler_y.fit(Y_train)
scaler_x.fit(X_test)
scaler_y.fit(Y_test)

# apply scaler to data set
xtrain_scale = scaler_x.transform(X_train)
ytrain_scale = scaler_y.transform(Y_train)

xtest_scale = scaler_x.transform(X_test)
ytest_scale = scaler_y.transform(Y_test)

# fit and transform in one line
# scaler_x.fit_transform(X_train)

# remeber to inverse the scaling on model output
# scaler_x.inverse_transform(xtest_scale)

### Model 1 RandomForest Regressor

#### Train RandomForest

In [9]:
# define Random Forest Regressor
rf_regressor_milk = RandomForestRegressor(random_state=2021)

# define list of Parameters
params_rf_milk = {'n_estimators':[100,500,800],
                  'criterion':['squared_error', 'absolute_error', 'poisson'],
                  'max_features':["auto", "sqrt", "log2"],
                  "bootstrap": [True, False]
                   }

# Hyper parameter tuning via Grid Search Cross Validation 
grid_rf_milk = GridSearchCV(estimator= rf_regressor_milk,
                          param_grid= params_rf_milk,
                          n_jobs=-1,
                          cv=5
                     )

# Fit the grid to scaled data
grid_rf_milk.fit(xtrain_scale,ytrain_scale.reshape(-1))

# print best training model & R squared score
print('Best training model ',grid_rf_milk.best_estimator_)
print('Best training model score, coefficient of determination R squared', grid_rf_milk.best_score_)

Best training model  RandomForestRegressor(criterion='absolute_error', n_estimators=500,
                      random_state=2021)
Best training model score, coefficient of determination R squared 0.034406025017006604


#### Predict RandomForest

In [10]:
# Predict Milk Production and unscale back to original values
y_predict = scaler_y.inverse_transform(grid_rf_milk.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n',y_predict)
print('actual milk production values \n',Y_test)

# Calculate Mean Absolute Error
MAE_rf = mean_absolute_error(Y_test,y_predict)
#print(MAE_rf)

# add model score to Score Dataframe
df_score = pd.DataFrame(data={'Model':'RandomForest',
                           'Score MAE':MAE_rf},index=['Model 1'])

print(df_score)

predicted milk production values 
 [[46.1244    ]
 [46.1494    ]
 [46.81763226]
 [60.703     ]
 [44.0322    ]
 [38.5768    ]
 [55.66569032]]
actual milk production values 
 [[36.8]
 [39.3]
 [25. ]
 [53.7]
 [27.9]
 [39.4]
 [49.5]]
                Model  Score MAE
Model 1  RandomForest   9.730789


### Model 2 XGBOOST Regressor

#### Train XGBOOST

In [11]:
# define XGBRegressor
xgb_regressor_milk = XGBRegressor(random_state=2021)

# define parameters space to loop over
params_xgb_milk = {'n_estimators':[20,40,80,160,340,500],
             'max_depth':[3,6,9],
             'gamma':[0.01,0.1],
             'learning_rate':[0.001,0.01,0.1,1]
             }

# Hyper parameter tuning via Grid Search Cross Validation 
grid_xgb_milk = GridSearchCV(estimator=xgb_regressor_milk,
                     param_grid=params_xgb_milk,
                     #n_jobs=-1,
                     scoring=['r2','neg_root_mean_squared_error'],
                     refit= 'r2',
                     n_jobs=-1,
                     cv=5,
                     verbose=4
                     )

# fit grid to training scaled set
grid_xgb_milk.fit(xtrain_scale,ytrain_scale);


# print best training model & R squared score
print('Best training model ',grid_xgb_milk.best_estimator_)
print('Best model Parameters',grid_xgb_milk.best_params_)
print('Best training model score, coefficient of determination R squared', grid_xgb_milk.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20; neg_root_mean_squared_error: (test=-0.447) r2: (test=-0.556) total time=   0.1s
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40; neg_root_mean_squared_error: (test=-0.316) r2: (test=-1.202) total time=   0.1s
[CV 5/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40; neg_root_mean_squared_error: (test=-1.452) r2: (test=-0.512) total time=   0.1s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80; neg_root_mean_squared_error: (test=-0.430) r2: (test=-0.438) total time=   0.2s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=160; neg_root_mean_squared_error: (test=-0.758) r2: (test=-4.159) total time=   0.4s
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=340; neg_root_mean_squared_error: (test=-0.244) r2: (test=-0.306) total time=   0.7s

[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20; neg_root_mean_squared_error: (test=-0.784) r2: (test=-0.725) total time=   0.1s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40; neg_root_mean_squared_error: (test=-0.772) r2: (test=-0.672) total time=   0.1s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80; neg_root_mean_squared_error: (test=-0.756) r2: (test=-0.604) total time=   0.2s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=160; neg_root_mean_squared_error: (test=-0.726) r2: (test=-0.478) total time=   0.4s
[CV 5/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=160; neg_root_mean_squared_error: (test=-1.415) r2: (test=-0.436) total time=   0.4s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=340; neg_root_mean_squared_error: (test=-0.370) r2: (test=-0.064) total time=   0.8s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_

[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20; neg_root_mean_squared_error: (test=-0.813) r2: (test=-4.938) total time=   0.1s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40; neg_root_mean_squared_error: (test=-0.805) r2: (test=-4.815) total time=   0.1s
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80; neg_root_mean_squared_error: (test=-0.301) r2: (test=-0.998) total time=   0.2s
[CV 5/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80; neg_root_mean_squared_error: (test=-1.439) r2: (test=-0.486) total time=   0.2s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=160; neg_root_mean_squared_error: (test=-0.410) r2: (test=-0.307) total time=   0.4s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=340; neg_root_mean_squared_error: (test=-0.706) r2: (test=-3.475) total time=   0.8s
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_e

[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20; neg_root_mean_squared_error: (test=-0.325) r2: (test=-1.320) total time=   0.1s
[CV 5/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=20; neg_root_mean_squared_error: (test=-1.458) r2: (test=-0.526) total time=   0.1s
[CV 4/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=40; neg_root_mean_squared_error: (test=-0.441) r2: (test=-0.515) total time=   0.1s
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=80; neg_root_mean_squared_error: (test=-0.789) r2: (test=-4.582) total time=   0.2s
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=160; neg_root_mean_squared_error: (test=-0.279) r2: (test=-0.710) total time=   0.4s
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=340; neg_root_mean_squared_error: (test=-0.670) r2: (test=-0.259) total time=   0.8s
[CV 5/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_e

Best training model  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0.1, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=160, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=2021,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
Best model Parameters {'gamma': 0.1, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 160}
Best training model score, coefficient of determination R squared -0.15271282786267798


#### Predict XGBOOST

In [12]:
# Predict Milk Production and unscale back to original values
y_predict = scaler_y.inverse_transform(grid_xgb_milk.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n',y_predict)
print('actual milk production values \n',Y_test)

# Calculate Mean Absolute Error
MAE_xgb = mean_absolute_error(Y_test,y_predict)
#print(MAE_xgb)

# add model score to Score Dataframe
df_score = pd.DataFrame(data = {'Model':['RandomForest','XGBOOST'],
                                'Score MAE': [MAE_rf,MAE_xgb]},
                        index=['Model 1','Model 2'])

print(df_score)

predicted milk production values 
 [[45.5861  ]
 [45.99256 ]
 [50.727455]
 [59.506348]
 [50.727455]
 [35.886517]
 [48.685654]]
actual milk production values 
 [[36.8]
 [39.3]
 [25. ]
 [53.7]
 [27.9]
 [39.4]
 [49.5]]
                Model  Score MAE
Model 1  RandomForest   9.730789
Model 2       XGBOOST  10.595393


In [13]:
# write the Grid Search results to csv to choose best model with least resource consumption

#GS_xgb_df_milk = pd.DataFrame(GS_xgb_milk.cv_results_)
#GS_xgb_df_milk = GS_xgb_df_milk.sort_values('rank_test_r2')
#GS_xgb_df_milk.to_csv('./../artifacts/grid-search-xgb-milk-results.csv')

## ANN Artificial Neural Network

#### Training & Keras Parameter Tuning

In [14]:
# Define ANN model with Hyper paramter variable 
def build_model(hp):
    model= keras.Sequential()
    for i in range(hp.Int('num_layers',2,23)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                           min_value=23,
                                           max_value=600,
                                           step=32),
                              activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
        return model
    
# create a directory to store each iteration of modelling
tuner = RandomSearch(
        build_model,
        objective='val_mean_absolute_error',
        max_trials=5,
        executions_per_trial=3,
        directory='ANN-tuner',
        project_name='Milk production')

# Defined parameter space to search in
tuner.search_space_summary()

# train trial models and compare with validation set
tuner.search(xtrain_scale,ytrain_scale,epochs=50,validation_data=(xtest_scale,ytest_scale))

# print best 10 models according to val_mean_absolute_error
print('\n')
tuner.results_summary()

# get best model from training trials
bestANNModel = tuner.get_best_models(num_models=1)[0]

# fit best model to training scaled data and scaled test data
bestANNModel.fit(xtrain_scale,ytrain_scale,epochs=50,validation_data=(xtest_scale,ytest_scale))

Trial 5 Complete [00h 00m 12s]
val_mean_absolute_error: 0.38478166858355206

Best val_mean_absolute_error So Far: 0.2196421374877294
Total elapsed time: 00h 00m 58s
INFO:tensorflow:Oracle triggered exit


Results summary
Results in ANN-tuner/Milk production
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 5
units_0: 55
learning_rate: 0.01
Score: 0.2196421374877294
Trial summary
Hyperparameters:
num_layers: 3
units_0: 215
learning_rate: 0.0001
Score: 0.34854452808698017
Trial summary
Hyperparameters:
num_layers: 22
units_0: 311
learning_rate: 0.0001
Score: 0.38017324606577557
Trial summary
Hyperparameters:
num_layers: 15
units_0: 279
learning_rate: 0.0001
Score: 0.383053998152415
Trial summary
Hyperparameters:
num_layers: 4
units_0: 119
learning_rate: 0.0001
Score: 0.38478166858355206
Epoch 1/50
1/1 [==============================] - 1s 642ms/step - loss: 0.4311 - mean_absolute_error: 0.4311 - val_loss: 0.2170 -

1/1 [==============================] - 0s 43ms/step - loss: 0.2293 - mean_absolute_error: 0.2293 - val_loss: 0.3286 - val_mean_absolute_error: 0.3286
Epoch 47/50
1/1 [==============================] - 0s 37ms/step - loss: 0.2249 - mean_absolute_error: 0.2249 - val_loss: 0.3459 - val_mean_absolute_error: 0.3459
Epoch 48/50
1/1 [==============================] - 0s 35ms/step - loss: 0.2247 - mean_absolute_error: 0.2247 - val_loss: 0.3211 - val_mean_absolute_error: 0.3211
Epoch 49/50
1/1 [==============================] - 0s 57ms/step - loss: 0.2189 - mean_absolute_error: 0.2189 - val_loss: 0.3000 - val_mean_absolute_error: 0.3000
Epoch 50/50
1/1 [==============================] - 0s 42ms/step - loss: 0.2203 - mean_absolute_error: 0.2203 - val_loss: 0.3012 - val_mean_absolute_error: 0.3012


In [15]:
# Predict Milk Production and unscale back to original values
y_predict = scaler_y.inverse_transform(bestANNModel.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n',y_predict)
print('actual milk production values \n',Y_test)

# Calculate Mean Absolute Error
MAE_ANN = mean_absolute_error(Y_test,y_predict)
#print(MAE_xgb)

# add model score to Score Dataframe
df_score = pd.DataFrame(data = {'Model':['RandomForest','XGBOOST','ANN'],
                                'Score MAE': [MAE_rf,MAE_xgb,MAE_ANN]},
                        index=['Model 1','Model 2','Model 3'])

print(df_score)

predicted milk production values 
 [[54.641827]
 [46.304825]
 [30.681519]
 [61.56327 ]
 [37.969273]
 [44.04872 ]
 [56.90565 ]]
actual milk production values 
 [[36.8]
 [39.3]
 [25. ]
 [53.7]
 [27.9]
 [39.4]
 [49.5]]
                Model  Score MAE
Model 1  RandomForest   9.730789
Model 2       XGBOOST  10.595393
Model 3           ANN   8.645012


# Pickle file
    Save trained model into binary pickle file to use the model later with new input data from web app

In [16]:
# Dump/write Scaler into binary pickle
pickle.dump(scaler_x,open('./../artifacts/pkl_scaler_x','wb'))

# Read pickle file into variable to use scaler
scaler_x_pkl_ann = pickle.load(open('./../artifacts/pkl_scaler_x','rb'))

# Dump/write Scaler into binary pickle
pickle.dump(scaler_y,open('./../artifacts/pkl_scaler_y','wb'))

# Read pickle file into variable to use scaler
scaler_y_pkl_ann = pickle.load(open('./../artifacts/pkl_scaler_y','rb'))

In [17]:

# Dump/write model into binary pickle file in the current notebook directory
pickle.dump(bestANNModel,open('./../artifacts/pkl_ann_milk','wb'))

# Read pickle file into variable to use model
model_pkl_ann = pickle.load(open('./../artifacts/pkl_ann_milk','rb'))


## Example using pickle file with saved ANN model

# take input from source as array
data_input_from_webapp = np.array([ 357.3,  362.5,  172. , 1440.2, 2136.5])

# scale input with same scaler as used in model
scale_data_from_webapp = scaler_x.transform(data_input_from_webapp.reshape(1, -1))

# predict scaled value
scaled_prediction = bestANNModel.predict(scale_data_from_webapp)

# descale prediction back to normal value
prediction = scaler_y.inverse_transform(scaled_prediction)
print('\n Expected Milk Production is ',prediction[0][0] )

2022-01-23 20:20:52.949246: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://dd48629e-37e5-4c70-8d36-777b7b4c6a5b/assets

 Expected Milk Production is  76.89205


# Next step Model Deployment